In [1]:
# we will train a conditional wgan on svhn dataset
# we will use the gradient penalty to stabilize the training


In [1]:
model_name = "CWGAN_GP0"

# imports

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# hyperparameters for the dataset and the MOdel

In [3]:
# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 3
Z_DIM = 100
NUM_EPOCHS = 1
FEATURES_CRITIC = 16
FEATURES_GEN = 16
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10

# prepare the dataset

### we will use SVHN dataset for this example
### we will combine the train, test and extra datasets to make a bigger dataset

In [4]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
    ]
)

In [5]:
#get the dataset
#train part of svhn
train_dataset = datasets.SVHN(root="dataset_svhm/", split='train', transform=transforms, download=True)
#test part of svhn
test_dataset = datasets.SVHN(root="dataset_svhm/", split='test', transform=transforms, download=True)
#extra part of svhn
extra_dataset = datasets.SVHN(root="dataset_svhm/", split='extra', transform=transforms, download=True)
#concatenate the train, test and extra dataset
dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset, extra_dataset])
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)


Using downloaded and verified file: dataset_svhm/train_32x32.mat
Using downloaded and verified file: dataset_svhm/test_32x32.mat


In [ ]:
#print the total number of images in the dataset
print(len(dataset))
# print the shape of the images
print(dataset[0][0].shape)
# print the label of the image
print(dataset[0][1])

630420
torch.Size([3, 64, 64])
1


# Model


## generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self.block_architecture_generator(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self.block_architecture_generator(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self.block_architecture_generator(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self.block_architecture_generator(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def block_architecture_generator(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self.block_architecture_critic(features_d, features_d * 2, 4, 2, 1),
            self.block_architecture_critic(features_d * 2, features_d * 4, 4, 2, 1),
            self.block_architecture_critic(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
        )

    def block_architecture_critic(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)

--------------------------------

# model initialization

In [ ]:

def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)



In [ ]:
# initialize gen and critic
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_CRITIC).to(device)
initialize_weights(gen)
initialize_weights(critic)

### initialize optimizer

In [ ]:
# initializate optimizer
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))
opt_critic = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

### inintialize tensorboard

In [ ]:
# for tensorboard plotting
fixed_noise = torch.randn(100, Z_DIM, 1, 1).to(device)
#plot loss of generator and critic
writer_loss = SummaryWriter(f"runs/"+model_name+"/loss")
writer_real = SummaryWriter(f"logs/"+model_name+"/real")
writer_fake = SummaryWriter(f"logs/"+model_name+"/fake")

--------------------------------

## start training

In [ ]:


gen.train()
critic.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(128, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)

#### gradient penalty function for WGAN-GP

In [ ]:
def gradient_penalty(critic, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

In [ ]:
step = 0

for epoch in range(NUM_EPOCHS):
    #we will track the total loss of the generator and critic for each epoch over the entire dataset
    #initialize the total loss of the generator and critic for each epoch to 0
    total_loss_gen = 0
    total_loss_critic = 0
    #move these to device

    #have no gradient for these losse
    # total_loss_gen = torch.no_grad()
    # total_loss_critic = torch.no_grad()
    
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        cur_batch_size = real.shape[0]

        # Train Critic: max E[critic(real)] - E[critic(fake)]
        # equivalent to minimizing the negative of that
        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn(cur_batch_size, Z_DIM, 1, 1).to(device)
            fake = gen(noise)
            critic_real = critic(real).reshape(-1)
            critic_fake = critic(fake).reshape(-1)
            gp = gradient_penalty(critic, real, fake, device=device)
            #LOSS for critic with gradient penalty
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake)) + LAMBDA_GP * gp
            )
            
            
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()
            
        #trained critic
        

        # Train Generator: max E[critic(gen_fake)] <-> min -E[critic(gen_fake)]
        gen_fake = critic(fake).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        #update the total loss of the generator and critic for each batch in the epoch
        #add just the value no gradients
        #have no gradient for these losse
        #add just the value no gradientsfrom the loss_gen tensor
      
        with torch.no_grad():
            total_loss_gen += loss_gen.item()
            total_loss_critic += loss_critic.item()
        
        # Print losses occasionally and print to tensorboard in a batch
        if batch_idx % 100 == 0 and batch_idx > 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            )
        
        

        # Print losses occasionally and print to tensorboard per epoch
        if batch_idx % 100 == 0 and batch_idx > 0:
        
            ## PRINT for few epochs %10
            # print(
            #     f"Epoch [{epoch}/{NUM_EPOCHS}]  \
            #         Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            # )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:100], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:100], normalize=True)

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
                
                #AVERAGE LOSS
                
                # #get average loss of generator and critic for each epoch
                # avg_loss_gen = total_loss_gen / len(loader)
                # avg_loss_critic = total_loss_critic / len(loader)
                # #write loss to tensorboard
                # writer_loss.add_scalar("Generator loss Epoch", avg_loss_gen, global_step=step)
                # writer_loss.add_scalar("Critic loss Epoch", avg_loss_critic, global_step=step)
                
                #AVERAGE LOSS
                
                
                #BATCH LOSS
                
                #write gen_loss and critic_loss to tensorboard
                writer_loss.add_scalar("Generator loss Batch", loss_gen, global_step=step)
                writer_loss.add_scalar("Critic loss Batch", loss_critic, global_step=step)
                
                #BATCH LOSS
                
                
                # we will plot the gradient penalty
                writer_loss.add_scalar("GP", gp, global_step=step)
                #we will analyze for vanishing gradient
                writer_loss.add_scalar("Critic Real", critic_real.mean(), global_step=step)
                writer_loss.add_scalar("Critic Fake", critic_fake.mean(), global_step=step)
                #get the gradient of the critic for the parameters weights of first layer
                #we will write the norm of the gardient of weights of the first layer of the critic
                for name, param in critic.named_parameters():
                    if name == "disc.0.weight":
                        writer_loss.add_scalar("Critic Gradient w.r.t 1st layer", param.grad.norm(), global_step=step)
                    #also plot the norm of gradient of 2nd layer
                    elif name == "disc.2.0.weight":
                        writer_loss.add_scalar("Critic Gradient w.r.t 2nd layer", param.grad.norm(), global_step=step)
                    

            step += 1
        
        #save the trained model
    torch.save(gen.state_dict(), "trained_models/"+model_name+"/gen.pth")
    torch.save(critic.state_dict(), "trained_models/"+model_name+"/critic.pth")
    
    
    


Epoch [0/1] Batch 100/9851                   Loss D: -33.5960, loss G: 155.4140
Epoch [0/1] Batch 200/9851                   Loss D: -30.5077, loss G: 153.7208
Epoch [0/1] Batch 300/9851                   Loss D: -21.3123, loss G: 168.7093
Epoch [0/1] Batch 400/9851                   Loss D: -13.6487, loss G: 162.9595


In [ ]:
#print critic model
print(critic)

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(128, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)


In [ ]:

#loop over gradients of all layers of critic
for name, param in critic.named_parameters():
    #print the gradient of the layer
    print(name, param.grad)
#

disc.0.weight tensor([[[[ 7.8407e-01,  7.4738e-01,  8.5062e-01,  9.4985e-01],
          [ 7.0284e-01,  7.9309e-01,  8.6797e-01,  8.9328e-01],
          [ 8.5218e-01,  7.5827e-01,  9.3687e-01,  9.7449e-01],
          [ 7.3868e-01,  7.1442e-01,  8.7685e-01,  9.5196e-01]],

         [[ 6.2477e-01,  6.6376e-01,  6.0109e-01,  7.0282e-01],
          [ 4.6417e-01,  6.0043e-01,  5.7597e-01,  7.4010e-01],
          [ 5.5329e-01,  5.9842e-01,  7.3386e-01,  7.9908e-01],
          [ 4.3417e-01,  5.3281e-01,  5.7862e-01,  5.7864e-01]],

         [[ 3.6017e-01,  3.7759e-01,  4.3168e-01,  5.3915e-01],
          [ 1.9463e-01,  3.0793e-01,  3.3463e-01,  4.7954e-01],
          [ 2.7108e-01,  3.9397e-01,  4.4191e-01,  5.2744e-01],
          [ 1.9152e-01,  2.1039e-01,  3.9451e-01,  4.3631e-01]]],


        [[[-8.1605e-01, -7.0556e-01, -1.6455e+00, -1.3586e+00],
          [-1.1020e+00, -8.3130e-01, -1.7792e+00, -1.2738e+00],
          [-1.4418e+00, -1.3061e+00, -2.0517e+00, -1.8267e+00],
          [-1.9006

In [ ]:
for name, param in critic.named_parameters():
            if name == "0.weight":
                print("Critic Gradient w.r.t 1st layer", param.grad.norm())
                break

In [ ]:
#loop over disc.0.weight
for name, param in critic.named_parameters():
    if name == "disc.0.weight":
        #print the gradient of the layer
        print(name, param.grad)
        #print norm of the gradient of the layer
        print(torch.norm(param.grad))
        


disc.0.weight tensor([[[[ 7.8407e-01,  7.4738e-01,  8.5062e-01,  9.4985e-01],
          [ 7.0284e-01,  7.9309e-01,  8.6797e-01,  8.9328e-01],
          [ 8.5218e-01,  7.5827e-01,  9.3687e-01,  9.7449e-01],
          [ 7.3868e-01,  7.1442e-01,  8.7685e-01,  9.5196e-01]],

         [[ 6.2477e-01,  6.6376e-01,  6.0109e-01,  7.0282e-01],
          [ 4.6417e-01,  6.0043e-01,  5.7597e-01,  7.4010e-01],
          [ 5.5329e-01,  5.9842e-01,  7.3386e-01,  7.9908e-01],
          [ 4.3417e-01,  5.3281e-01,  5.7862e-01,  5.7864e-01]],

         [[ 3.6017e-01,  3.7759e-01,  4.3168e-01,  5.3915e-01],
          [ 1.9463e-01,  3.0793e-01,  3.3463e-01,  4.7954e-01],
          [ 2.7108e-01,  3.9397e-01,  4.4191e-01,  5.2744e-01],
          [ 1.9152e-01,  2.1039e-01,  3.9451e-01,  4.3631e-01]]],


        [[[-8.1605e-01, -7.0556e-01, -1.6455e+00, -1.3586e+00],
          [-1.1020e+00, -8.3130e-01, -1.7792e+00, -1.2738e+00],
          [-1.4418e+00, -1.3061e+00, -2.0517e+00, -1.8267e+00],
          [-1.9006

In [ ]:
#loop over disc.0.weight
for name, param in critic.named_parameters():
    if name == "disc.2.0.weight":
        #print the gradient of the layer
        print(name, param.grad)
        #print norm of the gradient of the layer
        print(torch.norm(param.grad))
        


disc.2.0.weight tensor([[[[-8.5868e-01,  9.2782e-01,  1.5244e+00,  9.7454e-01],
          [-4.3667e-01,  1.3204e+00,  2.1424e+00,  1.2145e+00],
          [ 1.5942e+00,  3.0790e+00,  3.8962e+00,  2.6808e+00],
          [ 1.5855e+00,  3.4778e+00,  3.6417e+00,  3.0243e+00]],

         [[-2.1592e-01,  1.9645e-01,  2.5137e-01,  5.4800e-01],
          [-4.6724e-01, -8.3227e-02,  2.2094e-02,  2.8649e-01],
          [-4.4857e-01, -4.7353e-02, -2.2078e-03,  3.0585e-01],
          [-5.6510e-01, -2.1597e-01, -9.8833e-02,  1.5396e-01]],

         [[-2.4463e-02,  2.0641e-01,  2.1972e-01,  3.0598e-01],
          [-1.6896e-01,  6.0944e-03,  5.5868e-02,  1.2908e-01],
          [-1.5119e-01,  8.0078e-02,  1.0144e-01,  1.9062e-01],
          [-2.3054e-01, -3.9305e-02, -3.4383e-02,  7.8567e-02]],

         ...,

         [[-1.5196e+00,  1.1001e+00,  1.8860e+00,  1.1905e+00],
          [-3.0266e-01,  1.9433e+00,  3.4264e+00,  1.8462e+00],
          [ 2.3714e+00,  4.7358e+00,  5.7475e+00,  4.3756e+00],
   

In [ ]:
# analyze gradient of first layer of critic
# for name, param in critic.named_parameters():
#     if name == "conv1.weight":
#         print("inside loop")
#         print(param.grad)
#         print("conv1.weight", param.grad, global_step=step)
#loop over parameters in critic[0].weight.grad
for i in range(0, len(critic[0].weight.grad)):
    for j in range(0, len(critic[0].weight.grad[i])):
        print("critic[0].weight.grad", critic[0].weight.grad[i][j])
        # ("critic[0].weight.grad", critic[0].weight.grad[i][j], global_step=step)

TypeError: 'Discriminator' object is not subscriptable

In [ ]:
#  sudo fuser -k /dev/nvidia
